# Scripting GIS with R

[Daniel Contreras has a chapter in Ben Marwick’s online book, ‘How To do Archaeological Science in R’ about using R as a GIS](https://benmarwick.github.io/How-To-Do-Archaeological-Science-Using-R/using-r-as-a-gis-working-with-raster-and-vector-data.html). We’re going to follow along with that chapter, but I have modified the code slightly so that it will run in our lab bench.  

## You can't run this notebook unless you have the R kernel installed

Back in week 1, I mentioned that our labbench had to be equipped with certain other tools so that your computer would understand the code we're going to use. This is one of those moments. Here, we need an 'R kernel' installed so that the machine understands the R statistical programming language. Make sure to run [[r_kernel_install.ipynb]] first, before you go any further. After you run that computational notebook successfully, **close and then reopen** Jupyter.

When you reopen jupyterlab desktop (our labbench), and then open _this_ file, you should see in the top-right of the interface a capital letter R:

![](r-is-running.png)

Note the empty circle. That means the kernel is idle; when it's doing something, it'll be full.

## The Task

This notebook is derived from the work of Daniel Contreras [in 'How to do Archaeological Science Using R](https://benmarwick.github.io/How-To-Do-Archaeological-Science-Using-R/using-r-as-a-gis-working-with-raster-and-vector-data.html). I'll just quote Contreras:

>A common GIS task in archaeology is that of relating raster and vector data - e.g., relating site locations to environmental variables such as elevation, slope, and aspect. It’s common to do this by calculating values for the point locations in a shapefile of sites, and often of interest to compare environmental variables across some aspect of site variability - function, time period, size, etc. Here we’ll take a slightly more robust approach that looks at the areas around sites rather than their precise locations, calculating a buffer around each site location and then averaging the values of multiple environmental variables within those buffers. For simplicity we’ll use a DEM and rasters of slope and aspect that we can derive from it, but any other raster data could also be employed. Using a fictionalized dataset for a small area in Provence, we’ll take a look at some tools for exploring spatial data, and show that the areas settled in the Early Iron Age and the Gallo-Roman Period were significantly distinct with respect to some variables, but similar in others. <Br><br> We might hypothesize, for instance, that the Roman colonization of the area was based on cereal agriculture production for export, and that as a result Gallo-Roman period settlement would have prioritized relatively low, flat land appropriate for such activity. We could test this hypothesis by comparing the areas occupied in the two periods with respect to elevation and slope.

## Let's get what we need

In [ ]:
# this might take a couple of minutes to install.
# go get a cup of coffee

install.packages(c("sf", "terra"), type = "binary")
install.packages(c("protolite"), type = "binary")
install.packages(c("geojsonio"))


Just as with Python, those packages bind together lower-level commands in complicated ways in order to enable new functions for us. In this case, these packages allow us to deal with the various kinds of data that a GIS might use. The next cell explicitly tells the kernel to check these packages when encountering commands in order to know what to do.

In [ ]:
require(terra)     # for dealing with raster data
require(sf)        # for dealing with vector data
require(curl) 
require(lattice)
require(gridExtra)

## you'll get some warning messages, but that's ok, you can ignore them.

A geographic information system uses the metaphor of 'layers' to manage all kinds of geospatial data. By stacking up layers, we can then do computations down _through_ the layers. We might want to know if archaeological sites are associated with a particular kind of geomorphology, for instance. A GIS allows us to see how an archaeological layer containing specific kinds of features _intersects_ with a specific kind of soil.

But first we need to get data. NASA has made available 'digital elevation models' from 'shuttle radar topography' missions ([details here](https://www.earthdata.nasa.gov/data/instruments/srtm)) at a variety of resolutions. We're going to get that data.


In [ ]:
# Load data
# Import 30m DEM from NASA SRTM data in geotiff format

# Download the DEM file
x <- "https://raw.githubusercontent.com/benmarwick/How-To-Do-Archaeological-Science-Using-R/master/05_Contreras/demo_files/areaDEM.tif"
download.file(x, destfile = "areaDEM.tif", method = "curl")

The digital elevation model is a kind of image file, where the different colour pixel values equal different elevations. We're going to load that image into our notebook as a raster, and then match it ("project to") the real-world coordinate system UTM with WGS84 datum.

In [ ]:
# Load as raster using terra
areaDEM <- rast("areaDEM.tif")

# Project to UTM using terra
areaDEMutm <- project(areaDEM, "EPSG:32631") # EPSG:32631 = UTM Zone 31N


# Check the result
areaDEMutm

Now we'll load a number of vector files. In a GIS, these are called 'shapefiles' and they literally describe the shape of different kinds of information as geometric polygons or points. A shapefile comes with associated supporting files. All of them are necessary for the GIS to interpret things correctly. This will give us our archaeological site data.

In [ ]:
# Load vector data (sites)
# Download all shapefile components
ap1 <- "https://raw.githubusercontent.com/benmarwick/How-To-Do-Archaeological-Science-Using-R/master/05_Contreras/demo_files/areaPoints.shp"
ap2 <- "https://raw.githubusercontent.com/benmarwick/How-To-Do-Archaeological-Science-Using-R/master/05_Contreras/demo_files/areaPoints.dbf"
ap3 <- "https://raw.githubusercontent.com/benmarwick/How-To-Do-Archaeological-Science-Using-R/master/05_Contreras/demo_files/areaPoints.prj"
ap4 <- "https://raw.githubusercontent.com/benmarwick/How-To-Do-Archaeological-Science-Using-R/master/05_Contreras/demo_files/areaPoints.shx"

download.file(ap1, destfile = "areaPoints.shp", method = "curl")
download.file(ap2, destfile = "areaPoints.dbf", method = "curl")
download.file(ap3, destfile = "areaPoints.prj", method = "curl")
download.file(ap4, destfile = "areaPoints.shx", method = "curl")

Let's take a look at what we've got. 'EIA' = early iron age, 'GalRom' = Gallo-Roman. The early iron age predates the Gallo-Roman period, fyi.

In [ ]:
# Read shapefile using sf
sites <- st_read("areaPoints.shp")

In [ ]:
# Subset points to eliminate sites of uncertain date
sites_sub <- sites[sites$period == "EIA" | sites$period == "GalRom", ]

# Drop unused factor levels
sites_sub$period <- factor(sites_sub$period)

# Transform to UTM using sf
sites_sub_utm <- st_transform(sites_sub, crs = "EPSG:32631")  # EPSG:32631 = UTM Zone 31N

# If you want to plot with the DEM
plot(areaDEMutm, main = "DEM and Sites")
plot(st_geometry(sites_sub_utm), add = TRUE, col = "red", pch = 16)

**Does anything jump out at you?**

You might want to make a new markdown cell and record your thoughts!

## Derive new data from the DEM

In the next cell, we use the 'terrain' function with its options 'slope' and 'aspect' to calculate new information from the digital elevation model. 'Slope' is how steep the hill is, and 'aspect' is which direction (in degrees) that bit of land faces.

In [ ]:
# Calculate slope in degrees
area_slope <- terrain(areaDEMutm, v = "slope", unit = "degrees")

# Calculate aspect in degrees
area_aspect <- terrain(areaDEMutm, v = "aspect", unit = "degrees")

# Check the results
area_slope
area_aspect


In [ ]:
# Create a raster stack using terra
terrainstack <- c(areaDEMutm, area_slope, area_aspect)
names(terrainstack) <- c("elevation", "slope", "aspect")

# Check the stack
# This will give us some summary information for the three layers
terrainstack

### Plotting the Data
Now that we have that data for each pixel in our DEM, we can visualize those layers and see where our sites appear on them. We create a plot and set its parameters, and then we fill the plot (we plot inside the plot, ha) with three different visualizations, whose characteristics we set. 

In [ ]:
# Set up a large plot size and layout for single column + legend
options(repr.plot.width = 12, repr.plot.height = 15)

# Create layout: 3 rows, 2 columns
# Column 1 (plots): rows 1-3, Column 2 (legend): rows 1-3 combined
layout(matrix(c(1, 4,
                2, 4, 
                3, 4), 
              nrow = 3, ncol = 2, byrow = TRUE),
       widths = c(3, 1))  # Make plots wider than legend area

# Set margins for the plot panels
par(mar = c(4, 4, 3, 1))  # Reduced right margin since legend is separate

# Panel 1: Elevation
plot(areaDEMutm, main = "Elevation", col = terrain.colors(50))
points(coords_utm, 
       pch = ifelse(sites_sub_utm$period == "EIA", 18, 20),
       col = ifelse(sites_sub_utm$period == "EIA", "red", "blue"),
       cex = 0.8)

# Panel 2: Slope
plot(area_slope, main = "Slope (degrees)", col = terrain.colors(50))
points(coords_utm, 
       pch = ifelse(sites_sub_utm$period == "EIA", 18, 20),
       col = ifelse(sites_sub_utm$period == "EIA", "red", "blue"),
       cex = 0.8)

# Panel 3: Aspect
plot(area_aspect, main = "Aspect (degrees)", col = terrain.colors(50))
points(coords_utm, 
       pch = ifelse(sites_sub_utm$period == "EIA", 18, 20),
       col = ifelse(sites_sub_utm$period == "EIA", "red", "blue"),
       cex = 0.8)

# Panel 4: Legend (spanning all three rows)
par(mar = c(0, 0, 0, 0))  # No margins for legend panel
plot.new()
legend("center", 
       legend = c("EIA", "GalRom"), 
       pch = c(18, 20), 
       col = c("red", "blue"),
       bg = "white",
       cex = 1.2,
       pt.cex = 1.5,
       box.lwd = 1.5)

# Reset plot parameters
par(mfrow = c(1, 1))
layout(1)  # Reset layout. You have to do this or R gets cranky the next time you plot something.

**What do you see?**
You might want to make a new markdown cell and record your thoughts! Remember, our original hypothesis was that Gallo-Roman period settlement would have prioritized relatively low, flat land appropriate for such activity. 

Also, here's how you might want to change some of the point elements:

`pch = ifelse(sites_sub_utm$period == "EIA", 18, 20)`
+ `pch` = "plotting character" - controls the shape of the points
+ Uses `ifelse()` to conditionally assign shapes based on the "period" column
   + If period is "EIA" → uses pch 18 (filled diamond ♦)
   + If period is anything else → uses pch 20 (filled circle ●)

`col = ifelse(sites_sub_utm$period == "EIA", "red", "blue")`
+ col = color of the points
+ Again uses conditional logic based on period
   + EIA sites → red points
   + Non-EIA sites → blue points

`cex = 0.8`
`cex` = "character expansion" - controls the size of the points
   + 0.8 = makes points 80% of their default size (slightly smaller)
   + Values > 1 make points bigger, < 1 make them smaller


## Box Plotting and Exploratory Data Analysis

Picking things out by eye won't get you very far, in the long run. We're going to make some boxplots for the three different elements 'elevation', 'slope', and 'aspect'. A box plot is a way of summarizing the data so that you can see where the majority of your data lies, as well as its distribution and outliers. The 'notch' in the box plot will indicate roughly the 95% confidence interval for the median values.

In [ ]:
# Extract values around sites using terra
# Convert sf object to SpatVector for terra
sites_vect <- vect(sites_sub_utm)

# Extract mean values within 250m buffer
# We chose '250 m' rather arbitrarily. It might be that that's the _wrong_ value. 
# What would make sense archaeologically? How far could a person walk in a day, in a morning, in an hour?
# One might want to read some of the ancient Roman agronomists to get a sense of what value makes sense.
# Even though we're doing quantitative analysis here, the choices we make need to be grounded in an argument!
# After you've run through this notebook, you might want to come back and try some different values here.
sites_vals <- extract(terrainstack, sites_vect, buffer = 250, fun = mean, na.rm = TRUE)

# Combine with original data
sites_data <- data.frame(
  period = sites_sub_utm$period,
  sites_vals[, -1]  # Remove ID column
)

# Check sample sizes
summary(sites_data$period)


In [ ]:
# Create boxplots using lattice
elevplot <- bwplot(elevation ~ period,
                   data = sites_data,
                   notch = TRUE,
                   pch = "|",
                   fill = "grey",
                   box.ratio = 0.25,
                   par.settings = list(
                     box.rectangle = list(
                       col = c("red", "blue"))),
                   ylab = "masl",
                   main = "Elevation",
                   scales = list(x = list(labels = c("Early Iron Age\n(n = 94)",
                                                     "Gallo-Roman\n(n = 491)")),
                                 rot = 60))

# Display elevation plot
elevplot

In [ ]:
# Slope boxplot
slopeplot <- bwplot(slope ~ period,
                    data = sites_data,
                    notch = TRUE,
                    pch = "|",
                    fill = "grey",
                    box.ratio = 0.25,
                    par.settings = list(
                      box.rectangle = list(
                        col = c("red", "blue"))),
                    ylab = "slope (degrees)",
                    main = "Slope",
                    scales = list(x = list(labels = c("Early Iron Age\n(n = 94)",
                                                      "Gallo-Roman\n(n = 491)")),
                                  rot = 60))

# Display slope plot
slopeplot

In [ ]:
# Aspect boxplot
aspectplot <- bwplot(aspect ~ period,
                     data = sites_data,
                     notch = TRUE,
                     pch = "|",
                     fill = "grey",
                     box.ratio = 0.25,
                     par.settings = list(
                       box.rectangle = list(
                         col = c("red", "blue"))),
                     ylab = "aspect (degrees)",
                     main = "Aspect",
                     scales = list(x = list(labels = c("Early Iron Age\n(n = 94)",
                                                       "Gallo-Roman\n(n = 491)")),
                                   rot = 60))

# Display aspect plot
aspectplot

## Let's plot those side by side

Because it'll make it easier to inspect and compare.

In [ ]:
# Combine all three plots
grid.arrange(elevplot, slopeplot, aspectplot, nrow = 1, ncol = 3)


So... what do you think, re our original hypothesis? What other kind of data might be pertinent? 

This only scratches the surface (and not all that deeply) of what you might do re GIS and spatial analysis. But notice what we're _not_ doing here: I'm not giving you piles of screenshots telling you to click here... now click here.... now click on this... oh but you've got an older version, so click _this_ instead... click. Instead, we have a single document, with actionable code, and you are able to re-run the same work that I have done on my machine.

Most of you are coming from the History department: imagine what _reproducible_ historical research might look like...